In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload

In [3]:
from convert_pc_to_usd import load_GDP,return_usd

In [4]:
GDP_file = 'C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\summary_all_data.xlsx'

In [5]:
GDP2 = load_GDP(GDP_file)

In [6]:
resilience_folder = "C:\\Users\\WB451855\OneDrive - WBG\\!infra_needs_data\\resilience\\"

In [7]:
inv_needs = pd.read_excel(resilience_folder+"power robcosts_Julie.xlsx","power_investment_needs")

In [8]:
extra_cost_exposure = pd.read_excel(resilience_folder+"power robcosts_Julie.xlsx","cost_increase_exposure")
extra_cost_all = pd.read_excel(resilience_folder+"power robcosts_Julie.xlsx","cost_increase_all")

In [ ]:
inv_needs = pd.read_excel(resilience_folder+"power robcosts_Julie.xlsx","power_investment_needs")

In [9]:
inv_needs.sample(5)

,Variable,Model,Region,Scenario,2015,2020,2025,2030
100,EnergyInv_gdp,AIM/CGE,R5OECD90+EU,NDC,0.008754,0.007833,0.010466,0.013996
139,EnergyInv_gdp,POLES,R5LAM,NDC,0.011877,0.011438,0.009803,0.010575
78,EnergyInv_gdp_increasedstandards,WITCH-GLOBIOM,R5LAM,2C,0.013644,0.008470,0.007525,0.007524
17,EnergyInv_gdp_increasedstandards,IMAGE,R5ASIA,CPol,0.026776,0.017124,0.012779,0.010101
129,EnergyInv_gdp,MESSAGEix-GLOBIOM,R5OECD90+EU,2C,0.004765,0.005166,0.004575,0.004107


In [10]:
inv_needs_base = inv_needs[inv_needs.Variable=="EnergyInv_gdp"]

In [11]:
inv_needs_base = inv_needs_base.set_index(["Model","Region","Scenario"]).drop("Variable",axis=1)
inv_needs_base.columns.name='year'
inv_needs_base = inv_needs_base.stack().to_frame().rename(columns={0:"divided_by_gdp"})

In [12]:
extra_cost_all = extra_cost_all.set_index(["Model","Region","Scenario"]).drop("Discounted",axis=1)
extra_cost_all.columns.name='year'
extra_cost_all = extra_cost_all.stack().to_frame().rename(columns={0:"cost_increase_all"})

In [13]:
extra_cost_exposure = extra_cost_exposure.set_index(["Model","Region","Scenario"]).drop("Discounted",axis=1)
extra_cost_exposure.columns.name='year'
extra_cost_exposure = extra_cost_exposure.stack().to_frame().rename(columns={0:"cost_increase"})

In [14]:
all_res = inv_needs_base.reset_index().merge(extra_cost_all.reset_index(),on=["Model","Region","Scenario","year"]).merge(extra_cost_exposure.reset_index(),on=["Model","Region","Scenario","year"])

In [15]:
all_res['divided_by_gdp_res'] = all_res['divided_by_gdp']*(1+all_res['cost_increase'])
all_res['divided_by_gdp_res_all'] = all_res['divided_by_gdp']*(1+all_res['cost_increase_all'])

In [16]:
all_res['cost_diff'] = all_res['divided_by_gdp_res']-all_res['divided_by_gdp']
all_res['cost_diff_all'] = all_res['divided_by_gdp_res_all']-all_res['divided_by_gdp']

In [17]:
all_res_av = all_res.set_index(["Model","Region","Scenario",'year']).mean(level=(["Model","Region","Scenario"]))

In [18]:
all_res_av.reset_index().Region.unique()

array(['R5ASIA', 'R5LAM', 'R5MAF', 'R5OECD90+EU', 'R5REF'], dtype=object)

In [19]:
all_res_av = all_res_av.reset_index()

all_res_av["region"] = all_res_av.Region.replace({"R5MAF":"Africa",\
                                      "R5LAM":"Latin America and Caribbean",\
                                      "R5REF":"Eastern Europe and Central Asia",\
                                      "R5ASIA":"Asia",\
                                      "R5OECD90+EU":"OECD"})

In [20]:
all_res_av_usd = return_usd(all_res_av,'region','cost_diff',["Model","Scenario"],GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [21]:
all_res_av_usd.set_index(['region',"Model","Scenario"]).sum(level=["Model","Scenario"]).cost_bUSD_disc_annual.describe()

count    18.000000
mean     16.420776
std       5.511063
min       8.989644
25%      13.089095
50%      15.832537
75%      19.561188
max      27.549049
Name: cost_bUSD_disc_annual, dtype: float64

In [22]:
all_res_av_usd_tot = all_res_av_usd.set_index(['region',"Model","Scenario"]).sum(level=["Model","Scenario"])

In [23]:
all_res_av_usd_tot.loc[all_res_av_usd_tot['cost_bUSD_disc_annual'].idxmin()]

cost_bUSD_disc           134.844660
cost_bUSD_disc_annual      8.989644
Name: (MESSAGEix-GLOBIOM, CPol), dtype: float64

In [24]:
all_res_av_usd_tot.loc[all_res_av_usd_tot['cost_bUSD_disc_annual'].idxmax()]

cost_bUSD_disc           413.235736
cost_bUSD_disc_annual     27.549049
Name: (REMIND-MAgPIE, 2C), dtype: float64

In [25]:
all_res_av_usd_all = return_usd(all_res_av,'region','cost_diff_all',["Model","Scenario"],GDP2,d=0.06,ssp='ssp2',wbregions=False)

In [38]:
all_res_av_usd_all_tot = all_res_av_usd_all.set_index(['region',"Model","Scenario"]).sum(level=["Model","Scenario"])

In [27]:
all_res_av_usd_all_tot.cost_bUSD_disc_annual.describe()

count     18.000000
mean     178.801973
std       60.102166
min       95.896689
25%      145.169809
50%      166.015545
75%      227.623981
max      296.489310
Name: cost_bUSD_disc_annual, dtype: float64

In [28]:
all_res_av[['cost_increase','cost_increase_all']].describe()

,cost_increase,cost_increase_all
count,90.000000,90.000000
mean,0.030363,0.294836
std,0.018624,0.022412
min,0.001410,0.198443
25%,0.022454,0.283698
50%,0.027092,0.295448
75%,0.038332,0.309101
max,0.067434,0.336481


In [29]:
all_res_av[['cost_increase','cost_increase_all']].mean()

cost_increase        0.030363
cost_increase_all    0.294836
dtype: float64

In [39]:
all_res_av_usd_all_tot = all_res_av_usd_all_tot.reset_index()
select = (all_res_av_usd_all_tot.Scenario=="2C")&((all_res_av_usd_all_tot.Model=="AIM/CGE")|(all_res_av_usd_all_tot.Model=="MESSAGEix-GLOBIOM")|(all_res_av_usd_all_tot.Model=="REMIND-MAgPIE"))
extract = all_res_av_usd_all_tot[select]

extract['exposed'] = 'no'

C:\ProgramData\Anaconda3\envs\Julie1\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [40]:
extract

,Model,Scenario,cost_bUSD_disc,cost_bUSD_disc_annual,exposed
0,AIM/CGE,2C,3609.918520,240.661235,no
6,MESSAGEix-GLOBIOM,2C,1513.794676,100.919645,no
12,REMIND-MAgPIE,2C,4447.339646,296.489310,no


In [41]:
all_res_av_usd_tot = all_res_av_usd_tot.reset_index()
select = (all_res_av_usd_tot.Scenario=="2C")&((all_res_av_usd_tot.Model=="AIM/CGE")|(all_res_av_usd_tot.Model=="MESSAGEix-GLOBIOM")|(all_res_av_usd_tot.Model=="REMIND-MAgPIE"))
extract2 = all_res_av_usd_tot[select]

extract2['exposed'] = 'yes'

C:\ProgramData\Anaconda3\envs\Julie1\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [43]:
extract.append(extract2).to_csv("three_scenarios_power_resilience.csv")